Follow along the example here: https://github.com/doolingdavidrs21/openalex-jamming-gpt4/blob/main/jamming2d.ipynb

In [1]:
from pyalex import (
    Works, Authors, Sources,
    Institutions, Concepts, Publishers, Funders
)
import pyalex
import pandas as pd
import numpy as np
pyalex.config.email = "david@rs21.io"

from flair.embeddings import DocumentPoolEmbeddings
from flair.data import Sentence
from flair.embeddings import SentenceTransformerDocumentEmbeddings

EMBEDDING_MODEL_1 = "all-mpnet-base-v2" 

# this one is also good: all-MiniLM-L6-v2
EMBEDDING_MODEL_2 = "all-MiniLM-L6-v2"
SENT_EMBEDDINGS_1 = SentenceTransformerDocumentEmbeddings(EMBEDDING_MODEL_1)
SENT_EMBEDDINGS_2 = SentenceTransformerDocumentEmbeddings(EMBEDDING_MODEL_2)
DOC_EMBEDDINGS= DocumentPoolEmbeddings([SENT_EMBEDDINGS_2])

import torch
from tqdm import tqdm
import yake
import umap.umap_ as umap
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture as GMM
import altair as alt
import math
import plotly.express as px
import textwrap

2023-10-13 15:22:43.881995: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 15:22:45.116003: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
search_term = 'hypersonic'
robot_concepts = Concepts().search_filter(display_name=search_term).get()
len(robot_concepts)
for i in range(len(robot_concepts)):
    id_, display_name = robot_concepts[i]['id'], robot_concepts[i]['display_name']
    print(id_, display_name)

https://openalex.org/C122824865 Hypersonic speed
https://openalex.org/C198813307 Hypersonic flight
https://openalex.org/C2984849654 Hypersonic flow
https://openalex.org/C11066151 Hypersonic wind tunnel


In [3]:
search_term = 'hypersonic'
hypersonic_concepts = Concepts().\
search_filter(display_name=search_term).get()

In [4]:
concepts = []
for i in range(len(hypersonic_concepts)):
    id_, display_name = hypersonic_concepts[i]['id'], hypersonic_concepts[i]['display_name']
    concepts.append((id_, display_name))
concepts

[('https://openalex.org/C122824865', 'Hypersonic speed'),
 ('https://openalex.org/C198813307', 'Hypersonic flight'),
 ('https://openalex.org/C2984849654', 'Hypersonic flow'),
 ('https://openalex.org/C11066151', 'Hypersonic wind tunnel')]

In [5]:
def process_works_list(worklist:list):
    """
    transforms the 
    works list into a dataframe.
    """
    abstracts_dict = {h["id"]:h["abstract"] for h in worklist}
    df = pd.DataFrame.from_records(worklist)
    del df['abstract_inverted_index']
    df['abstract'] = df['id'].map(abstracts_dict)
   # df['author_affils'] = df['authorships'].apply(get_authors_and_affils)
    return df

In [6]:
for i in range(len(hypersonic_concepts)):
    print(hypersonic_concepts[i]['id'], hypersonic_concepts[i]['works_count'])


https://openalex.org/C122824865 24944
https://openalex.org/C198813307 4462
https://openalex.org/C2984849654 3627
https://openalex.org/C11066151 3624


In [7]:
def get_concept_frame(concepts_list:list, i:int):
    """
    takes a list of Concepts() results and an index
    and forms the pagination object to retrive the 
    records
    """
    pager = Works().filter(publication_year='>2000',
    #concepts={"id":f"{concepts_list[i]['id']}"}).filter(authorships={"institutions":{"country_code":"CN"}}).\
    #paginate(per_page=200,n_max=None)
    concepts={"id":f"{concepts_list[i]['id']}"}).\
    paginate(per_page=200,n_max=None)
    df = pd.DataFrame()
    for page in tqdm(pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df

In [8]:
frames_list = []
for i in range(len(hypersonic_concepts)):
    df = get_concept_frame(hypersonic_concepts, i)
    frames_list.append(df)

76it [01:50,  1.46s/it]
14it [00:20,  1.47s/it]
8it [00:11,  1.38s/it]
12it [01:06,  5.57s/it]


In [9]:
len(frames_list)

4

In [10]:
dftop = pd.concat(frames_list,
                  ignore_index=True)
dftop.drop_duplicates(subset='id', keep='first', 
                      inplace=True)

dftop.set_index('id', inplace=True, drop=False)

dfall = dftop
print(dfall.shape)

dfall['content'] = dfall['title'] + ". " + dfall['abstract']

dfrecords = dfall[~dfall['content'].isna()].copy()

(16834, 41)


In [11]:
def get_keywords(text:str, top:int=7, stopwords=None):
    """
    takes a blob of text and 
    returns the top **top** 
    keywords as a list
    """
    kw_extractor = yake.KeywordExtractor(top=top, stopwords=stopwords)
    keywords = kw_extractor.extract_keywords(text)
    return [p[0] for p in keywords]

In [12]:
dfall.shape

(16834, 42)

In [13]:
def get_top_concepts(concept_list:list,score:float=.6):
    """
    takes a list of concept dictionaries 
    returns the top **top** display_names;
    concepts whose score is >= score
    """
    return [c['display_name'] for c in concept_list if c['score'] >= score]

In [14]:
dfrecords['keywords'] = dfrecords['content'].apply(get_keywords)
dfrecords['top_concepts'] = dfrecords['concepts'].apply(get_top_concepts)

In [15]:
texts = dfrecords['content'].str.lower().values.tolist()
#dfrecords.to_csv('jamming.csv')


In [16]:
def get_content_embeddings(dfrecords:pd.DataFrame) -> pd.DataFrame:
    """
    passes the preprocessed mitigation strings
    data through the embedding model to produce the vector
    space representation of each pet mitigation.
    """
    sent = Sentence("The grass is green.")
    DOC_EMBEDDINGS.embed(sent)
    texts = dfrecords["content"].str.lower().values.tolist()
    all_descriptions = np.empty((len(texts), len(sent.embedding)))
    for i in tqdm(range(len(texts))):
        sent = Sentence(texts[i])
        DOC_EMBEDDINGS.embed(sent)
        all_descriptions[i, :] = sent.embedding.cpu().numpy()
        # gc.collect()
        torch.cuda.empty_cache()
    dfcontentvectors = pd.DataFrame.from_records(all_descriptions, index=dfrecords.index)
    return dfcontentvectors

In [17]:
dfcontentvectors = get_content_embeddings(dfrecords)


100%|███████████████████████████████████████████████████████████████████| 12700/12700 [03:21<00:00, 62.97it/s]


In [18]:
#umap.UMAP?
N_COMPONENTS = 2 # can visualize this way
umap_reducer = umap.UMAP(n_components=N_COMPONENTS,
                       #  metric='euclidean')
                         random_state=1234,
                         metric='cosine')  # can experiment with this metric as well as the other 
# parameters
# to see what other literature is in the same information space, we need to keep this umap_reducer 
# object as well as the gmm model below.

# Apply UMAP to the vectorized strings
reduced_vectors = umap_reducer.fit_transform(dfcontentvectors.to_numpy())
dfreduced = pd.DataFrame.from_records(reduced_vectors, 
                index=dfcontentvectors.index)
dfreduced.columns = ['x','y']

# use hdbscan to cluster

In [19]:
import hdbscan

hdbscan_args = {'min_cluster_size': 15,
                            'metric': 'euclidean',
                            'cluster_selection_method': 'eom',
                            'cluster_selection_epsilon': 0.1
               }

cluster = hdbscan.HDBSCAN(**hdbscan_args).fit(dfreduced[['x','y']].to_numpy())

dfreduced['cluster'] = cluster.labels_
dfreduced['probability'] = cluster.probabilities_

dfpapers = dfrecords.merge(dfreduced, left_index=True,
                           right_index=True)

In [20]:
#help(dfpapers.explode)
del dfpapers['id']
dfstart = dfpapers.reset_index()
dfstart.head()

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,updated_date,created_date,abstract,content,keywords,top_concepts,x,y,cluster,probability
0,https://openalex.org/W2046971659,https://doi.org/10.2514/1.12596,Adaptive Sliding Mode Control Design for a Hyp...,Adaptive Sliding Mode Control Design for a Hyp...,2004,2004-09-01,{'openalex': 'https://openalex.org/W2046971659...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2023-10-13T13:13:49.855202,2016-06-24,A multi-input/multi-output adaptive sliding co...,Adaptive Sliding Mode Control Design for a Hyp...,"[Mode Control Design, Adaptive Sliding Mode, S...",[],12.240765,2.577896,10,1.000000
1,https://openalex.org/W2016332739,https://doi.org/10.2514/1.27830,Control-Oriented Modeling of an Air-Breathing ...,Control-Oriented Modeling of an Air-Breathing ...,2007,2007-05-01,{'openalex': 'https://openalex.org/W2016332739...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2023-10-03T01:09:35.774046,2016-06-24,Full simulation models for flexible air-breath...,Control-Oriented Modeling of an Air-Breathing ...,"[Air-Breathing Hypersonic Vehicle, hypersonic ...",[Aerospace engineering],10.622355,2.518020,10,1.000000
2,https://openalex.org/W1993626705,https://doi.org/10.2514/1.23370,Nonlinear Longitudinal Dynamical Model of an A...,Nonlinear Longitudinal Dynamical Model of an A...,2007,2007-03-01,{'openalex': 'https://openalex.org/W1993626705...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2023-09-30T01:23:09.714460,2016-06-24,Covers advancements in spacecraft and tactical...,Nonlinear Longitudinal Dynamical Model of an A...,"[Longitudinal Dynamical Model, Nonlinear Longi...","[Aerospace engineering, Spacecraft, Missile, H...",9.931794,3.204971,10,1.000000
3,https://openalex.org/W2162602330,https://doi.org/10.2514/1.39210,Nonlinear Robust Adaptive Control of Flexible ...,Nonlinear Robust Adaptive Control of Flexible ...,2009,2009-03-01,{'openalex': 'https://openalex.org/W2162602330...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2023-10-13T18:53:54.817551,2016-06-24,This paper describes the design of a nonlinear...,Nonlinear Robust Adaptive Control of Flexible ...,"[Flexible Air-Breathing Hypersonic, Nonlinear ...","[Hypersonic speed, Nonlinear system, Control t...",11.063985,2.276188,10,1.000000
4,https://openalex.org/W2166911839,https://doi.org/10.1146/annurev-fluid-122109-1...,Transition and Stability of High-Speed Boundar...,Transition and Stability of High-Speed Boundar...,2011,2011-01-21,{'openalex': 'https://openalex.org/W2166911839...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2023-10-11T06:31:18.558072,2016-06-24,This article reviews stability and laminar-tur...,Transition and Stability of High-Speed Boundar...,"[High-Speed Boundary Layers, Boundary Layers, ...","[Laminar-turbulent transition, Supersonic spee...",1.934812,8.734060,102,0.680959


In [21]:
dfstart.shape

(12700, 48)

In [22]:
dfbig = dfstart.explode(column='authorships')
dfbig.shape, dfstart.shape

((40276, 48), (12700, 48))

In [23]:
def add_extra_to_authorships(row: pd.DataFrame):
    """
    row[authorships] is a dictionary;
    add in the id key to that dictionary
    whose value is row[id]
    """
    complete_dict = row["authorships"]
   # assert type(complete_dict) == dict
    #print(type(complete_dict))
    if type(complete_dict) == dict:
        complete_dict["id"] = row["id"]
        complete_dict["x"] = row["x"]
        complete_dict["y"] = row["y"]
        complete_dict["cluster"] = row["cluster"]
        complete_dict["cluster_score"] = row["probability"]
        complete_dict["title"] = row["title"]
        complete_dict["abstract"] = row["abstract"]
        complete_dict["doi"] = row["doi"]
        complete_dict["publication_date"] = row["publication_date"]
        complete_dict["publication_year"] = row["publication_year"]
        complete_dict["grants"] = row["grants"]
        complete_dict["locations"] = row["locations"]
        return complete_dict
    else:
        return row["authorships"]

In [24]:

dfbig['big_authorships'] = dfbig.apply(add_extra_to_authorships, axis=1)

In [25]:
#dfbig['authorships'].tolist()
bigvals = dfbig['authorships'].tolist()

In [26]:
dictvals = [c for c in bigvals if type(c) != float]


In [27]:
dftriple = pd.json_normalize(dictvals,
                  record_path=['institutions'],
                  meta=['id','raw_affiliation_string','author_position', 'doi',
                        'title','abstract','publication_date', 'publication_year',
                        'grants','locations',
                        'is_corrresponding','x','y','cluster','cluster_score',
                       ['author','id'], ['author', 'display_name'],
                       ['author','orcid']],
                  errors='ignore',
                  sep='_',
                  meta_prefix='paper_',
                #  record_prefix='author_'
                 )

In [28]:
dftopics = dfcontentvectors.copy()
dftopics['cluster'] = dfpapers['cluster']
dfmeantopics = dftopics.groupby('cluster').mean().copy()
reduced_topics = umap_reducer.transform(dfmeantopics.to_numpy())
df_reduced_topics = pd.DataFrame.from_records(reduced_topics, 
                index=dfmeantopics.index)
df_reduced_topics.columns = ['x','y']
df_reduced_topics['topic'] = df_reduced_topics.index
df_reduced_topics.head()

def get_cluster_concepts(topic_num:int, n:int=20):
    """
    takes an integer topic_num corresponding to a 
    given topic number and
    returns the list of top n occuring concepts
    from the top_concept field
    """
    top_concepts = dfpapers[dfpapers['cluster'] == topic_num]['top_concepts'].tolist()
    flat_concepts = [item for sublist in top_concepts for item in sublist]
    concepts_dict = {c:flat_concepts.count(c) for c in flat_concepts}
    sorted_concepts = sorted(concepts_dict.items(), key=lambda x:x[1], reverse=True)
    return [c[0] for c in sorted_concepts][:n]

def get_yake_cluster_phrases(topic_num:int, n:int=20):
    """
    takes in an integer n corresponding
    to a given topic number and
    returns the list of keyphrases (TopicRank method)
    """
    documents = dfpapers[dfpapers['cluster'] == topic_num]['content'].tolist()
    topic_input = ". ".join(documents)
    #extractor = pke.unsupervised.TextRank()
    kw_extractor = yake.KeywordExtractor(top=n, stopwords=None)
    keywords = kw_extractor.extract_keywords(topic_input)
    #extractor.load_document(input=topic_input,
    #                    language='en',
    #                    normalization=None)

    #extractor.candidate_selection()

    #window = 2
    #use_stems = False
    #extractor.candidate_weighting(window=window,
    #                          use_stems=use_stems)
    #extractor.candidate_weighting()
    #threshold = 0.8
   # keyphrases = extractor.get_n_best(n=20, threshold=threshold)
    #keyphrases = extractor.get_n_best(n=n)
    return [p[0] for p in keywords]

wikiconcepts = df_reduced_topics['topic'].apply(get_cluster_concepts)

wikikeywords = df_reduced_topics['topic'].apply(get_yake_cluster_phrases)

dfpapers['id'] = dfpapers.index
dfinfo = dfpapers[['x','y','id','title','doi','cluster','grants',
                   'locations',
                 'publication_date','keywords','top_concepts']].copy()

centroids = dfinfo.groupby('cluster')[['x','y']].mean().copy()
centroids['concepts'] = wikiconcepts
centroids['cluster'] = centroids.index
centroids['keywords'] = wikikeywords

In [29]:
def wrap_it(x):
    return "<br>".join(textwrap.wrap(x, width=40))
   # return "<br>".join(textwrap.wrap(x.replace(r'\s+', ' '), width=40))

In [30]:
centroids['wrapped_keywords'] = centroids['keywords'].apply(str).apply(wrap_it)
centroids['wrapped_concepts'] = centroids['concepts'].apply(str).apply(wrap_it)

In [31]:
centroids.to_pickle('hypersoniccentroids2d.pkl')


In [32]:
dftriple.to_pickle('hypersonicdftriple2d.pkl')


In [33]:
def get_affils_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by id, ror sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg.groupby(['id','display_name','country_code',
                     'type'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [34]:
dv84, kw84 = get_affils_cluster_sort(dftriple, 1)
print(kw84)
dv84.head(10)

1
['wind tunnel test', 'wind tunnel', 'wind pressure', 'wind pressure coefficients', 'wind pressure distribution', 'wind', 'wind tunnel experiments', 'layer wind tunnel', 'tunnel test', 'wind load', 'wind tunnel study', 'RMS wind pressure', 'fluctuating wind pressure', 'CFD wind tunnel', 'numerical wind tunnel', 'wind pressure characteristics', 'wind tunnel measurements', 'wind tunnel models', 'wind loads', 'wind tunnel testing']


,,,,paper_cluster_score
id,display_name,country_code,type,
https://openalex.org/I116953780,Tongji University,CN,education,17.669269
https://openalex.org/I16609230,Hunan University,CN,education,12.176592
https://openalex.org/I889458895,University of Hong Kong,HK,education,11.130538
https://openalex.org/I158842170,Chongqing University,CN,education,11.0
https://openalex.org/I37461747,Wuhan University,CN,education,10.472673
https://openalex.org/I200769079,Hong Kong University of Science and Technology,CN,education,10.130538
https://openalex.org/I4210126275,Toda Kogyo (Japan),JP,company,10.0
https://openalex.org/I204983213,Harbin Institute of Technology,CN,education,10.0
https://openalex.org/I168719708,City University of Hong Kong,CN,education,10.0


In [35]:
dv84, kw84 = get_affils_cluster_sort(dftriple, 0)
print(kw84)
dv84.head(10)

0
['Hypersonic Phononic Crystals', 'Hypersonic', 'Brillouin light scattering', 'Hypersonic Phononic', 'Phononic Crystals', 'hypersonic surface phononic', 'hypersonic waves', 'Hypersonic Phononic Band', 'surface acoustic waves', 'Acoustic Waves', 'Brillouin light', 'surface phononic crystals', 'Phononic', 'hypersonic acoustic wave', 'acoustic', 'Brillouin scattering', 'hypersonic crystals', 'Brillouin', 'Waves', 'hypersonic acoustic']


,,,,paper_cluster_score
id,display_name,country_code,type,
https://openalex.org/I4210093216,Institut Català de Nanociència i Nanotecnologia,ES,facility,21.436175
https://openalex.org/I4210095245,Max Planck Institute for Polymer Research,DE,facility,14.178405
https://openalex.org/I95568926,Ioffe Institute,RU,facility,11.781572
https://openalex.org/I200332995,TU Dortmund University,DE,education,9.235005
https://openalex.org/I59411706,Adam Mickiewicz University in Poznań,PL,education,8.099491
https://openalex.org/I1294671590,French National Centre for Scientific Research,FR,government,7.958549
https://openalex.org/I142263535,University of Nottingham,GB,education,7.759876
https://openalex.org/I4210123471,"Institute of Electronics, Microelectronics and Nanotechnology",FR,facility,6.806951
https://openalex.org/I165932596,National University of Singapore,SG,education,6.01198


In [36]:
dfinfo = dfpapers[['x','y','id','title','abstract', 'doi','cluster','probability',
                 'publication_date','grants','locations',
                   'keywords','top_concepts']].copy()


In [37]:
pap_affils_dict = dftriple.groupby('paper_id')['paper_raw_affiliation_string'].\
apply(lambda x: ' | '.join(x.tolist()))

#pap_authors_dict = dftriple.groupby('paper_id')['paper_author_display_name'].\
#apply(lambda x: ' | '.join(x.tolist()))

In [38]:
pap_authors_dict = dftriple.groupby('paper_id')['paper_author_display_name'].apply(lambda x: x.values)

In [39]:
dfinfo['affil_list'] = pap_affils_dict
dfinfo['author_list'] = pap_authors_dict

In [40]:
dfinfo['affil_list'] = dftriple.groupby('paper_id')['paper_raw_affiliation_string'].\
apply(lambda x: x.tolist())

In [41]:
dfinfo['author_list'] =  dftriple.groupby('paper_id')['paper_author_display_name'].\
apply(lambda x: x.tolist())

In [42]:
dfinfo['wrapped_affil_list'] = dfinfo['affil_list'].apply(str).apply(wrap_it)
dfinfo['wrapped_author_list'] = dfinfo['author_list'].apply(str).apply(wrap_it)

In [43]:
dfinfo['wrapped_keywords'] = dfinfo['keywords'].apply(str).apply(wrap_it)


In [44]:
def get_source_name(loc_list):
    """
    grab the first item in the list;
    retturn the display name
    """
    try:
        primary = loc_list[0]
        return primary["source"]["display_name"]
    except:
        return None

def get_source_type(loc_list):
    """
    grab the first item in the list;
    return the source type
    """
    try:
        primary = loc_list[0]
        return primary["source"]["type"]
    except:
        return None

In [45]:
dfinfo["source"] = dfinfo["locations"].apply(get_source_name)
dfinfo["source_type"] = dfinfo["locations"].apply(get_source_type)

In [46]:
dfinfo.to_pickle('hypersonicdfinfo2d.pkl')


In [47]:
def get_funder_names(funder_list):
    """
    funder_list is a list of dictionaries
    with three keys; return the list of 
    unique **funder_display_name**
    values
    """
    try:
        funder_names = list(set([f['funder_display_name'] for f in funder_list]))
        return funder_names    
    except:
        return []

In [48]:

dftriple["source"] = dftriple["paper_locations"].apply(get_source_name)
dftriple["source_type"] = dftriple["paper_locations"].apply(get_source_type)
dftriple["funder_list"] = dftriple["paper_grants"].apply(get_funder_names)

In [49]:
dftriple.to_pickle('hypersonicdftriple2d.pkl')


In [50]:
dfinfo["funder_list"] = dfinfo["grants"].apply(get_funder_names)
dfinfo["wrapped_funder_list"] = dfinfo["funder_list"].apply(str).apply(wrap_it)

In [51]:
dfinfo.to_pickle('hypersonicdfinfo2d.pkl')


In [52]:
kw_dict = dfinfo['keywords'].to_dict()


In [53]:
sources_list = dftriple['source'].unique().tolist()
type(sources_list), len(sources_list)


(list, 1127)

In [54]:
def get_source_json(s:str):
    """
    s is an openalex Sources display_name
    return that Sources object
    """
    source_json = Sources().search_filter(display_name = s).get()
    a = source_json[0]['type']
    if "homepage_url" in source_json[0] and source_json[0]['homepage_url']:
        print(f"{s} has homepage_url and type {source_json[0]['type']}")
        return source_json[0]["homepage_url"]
    else:
        return None

In [58]:
sj0 = get_source_json(sources_list[65])
sj0

Proceedings of the Combustion Institute has homepage_url and type journal


'https://www.journals.elsevier.com/proceedings-of-the-combustion-institute'

In [59]:
sources_list[65]

'Proceedings of the Combustion Institute'

In [60]:
def get_display_page_dict(sl:list):
    """
    sl is a list of Sources display_name values
    returns the dictionary mapping
    display_names with homepage_url values."""
    mapping_dict = dict()
    for s in tqdm(sl):
        try:
            mapping_dict[s] = get_source_json(s)
        except:
            pass
    return mapping_dict

In [61]:
source_page_dict = get_display_page_dict(sources_list)

  0%|▏                                                                       | 2/1127 [00:01<09:43,  1.93it/s]

Journal of Spacecraft and Rockets has homepage_url and type journal


  0%|▏                                                                       | 3/1127 [00:01<09:23,  1.99it/s]

Annual Review of Fluid Mechanics has homepage_url and type journal


  0%|▎                                                                       | 4/1127 [00:02<09:16,  2.02it/s]

...

100%|████████████████████████████████████████████████████████████████████▊| 1123/1127 [08:34<00:01,  2.27it/s]

Japan architectural review has homepage_url and type journal


100%|████████████████████████████████████████████████████████████████████▊| 1124/1127 [08:34<00:01,  2.32it/s]

Sustainable Energy Technologies and Assessments has homepage_url and type journal


100%|█████████████████████████████████████████████████████████████████████| 1127/1127 [08:36<00:00,  2.18it/s]


In [62]:
import pickle

with open("source_page_dict.pkl", "wb") as f:
    pickle.dump(source_page_dict, f)

In [63]:
with open("source_page_dict.pkl", "rb") as f:
    source_dict = pickle.load(f)

In [64]:
def get_journals_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'journal'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    dv['journal'] = dv.index
    dv['hompage_url'] = dv['journal'].map(source_dict)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [65]:
dv, kw = get_journals_cluster_sort(dftriple, 2)
dv.head()

2


,paper_cluster_score,journal,hompage_url
source,,,
"Proceedings Of The Institution Of Mechanical Engineers, Part G: Journal Of Aerospace Engineering",5.521758,Proceedings Of The Institution Of Mechanical E...,NaN
Journal of Guidance Control and Dynamics,4.673638,Journal of Guidance Control and Dynamics,None
Aerospace,4.0,Aerospace,http://ieeexplore.ieee.org/xpl/mostRecentIssue...
Mathematical Problems in Engineering,4.0,Mathematical Problems in Engineering,https://www.hindawi.com/journals/mpe/
Proceedings of SPIE,4.0,Proceedings of SPIE,http://spie.org//x1848.xml?WT.svl=mddp2


In [66]:
affils_list = dftriple['display_name'].unique().tolist()
type(affils_list), len(affils_list)

affil_json = Institutions().filter(display_name = affils_list[1]).get()

def get_affil_json(s:str):
    """
    s is an openalex Sources display_name
    return that Sources object
    """
    affil_json = Institutions().search_filter(display_name = s).get()
    #a = source_json[0]['type']
    if "geo" in affil_json[0]:
       # print(f"{s} has homepage_url and type {source_json[0]['type']}")
        return affil_json[0]["geo"]["latitude"], affil_json[0]["geo"]["longitude"]
    else:
        return None, None
    
def get_display_geo_dict(sl:list):
    """
    sl is a list of Institution display_name values
    returns the dictionary mapping
    display_names with (latitude, longitude) values."""
    mapping_dict = dict()
    for s in tqdm(sl):
        try:
            mapping_dict[s] = get_affil_json(s)
        except:
            pass
    return mapping_dict

affil_geo_dict = get_display_geo_dict(affils_list)


import pickle

with open("affil_geo_dict.pkl", "wb") as f:
    pickle.dump(affil_geo_dict, f)

100%|█████████████████████████████████████████████████████████████████████| 2184/2184 [16:49<00:00,  2.16it/s]


In [67]:
for qc in hypersonic_concepts:
    print(qc)
    print("-------------")

{'id': 'https://openalex.org/C122824865', 'wikidata': 'https://www.wikidata.org/wiki/Q830787', 'display_name': 'Hypersonic speed', 'relevance_score': 11720.203, 'level': 2, 'description': 'speed that is highly supersonic', 'works_count': 24944, 'cited_by_count': 171791, 'summary_stats': {'2yr_mean_citedness': 1.7723240685984625, 'h_index': 127, 'i10_index': 4034}, 'ids': {'openalex': 'https://openalex.org/C122824865', 'wikidata': 'https://www.wikidata.org/wiki/Q830787', 'mag': '122824865', 'wikipedia': 'https://en.wikipedia.org/wiki/Hypersonic%20speed'}, 'image_url': 'https://upload.wikimedia.org/wikipedia/commons/5/55/X-43A_%28Hyper_-_X%29_Mach_7_computational_fluid_dynamic_%28CFD%29.jpg', 'image_thumbnail_url': 'https://upload.wikimedia.org/wikipedia/commons/thumb/5/55/X-43A_%28Hyper_-_X%29_Mach_7_computational_fluid_dynamic_%28CFD%29.jpg/100px-X-43A_%28Hyper_-_X%29_Mach_7_computational_fluid_dynamic_%28CFD%29.jpg', 'international': {'display_name': {'ar': 'هايبر سونيك', 'ast': 'Velo

In [70]:
from PIL import Image

In [69]:
import PIL